This project focuses on customer segmentation for a shopping mall using unsupervised machine learning techniques.

The objective is to group customers based on their annual income and spending score in order to support data-driven marketing strategies.

In this notebook, exploratory data analysis (EDA) is performed, followed by K-Means clustering. 
The optimal number of clusters is determined using the Elbow Method and Silhouette Score.

Finally, customer segments are interpreted from a business perspective, and actionable marketing strategies are proposed for each group.